# mmmeld Validation Tests

This notebook contains parameterized validation tests for mmmeld.py. We'll test various scenarios and rules using different combinations of media sources.

In [1]:
import subprocess
import os
from IPython.display import Video

def run_mmmeld(params):
    output_file = f"test_output_{params[params.index('--output') + 1]}"
    cmd = ["python", "mmmeld.py", "--autofill", "--cleanup"] + params + ["--output", output_file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
    return result.returncode == 0, output_file

def display_output(output_file):
    if os.path.exists(output_file):
        return Video(output_file)
    else:
        return f"Output file {output_file} not found."

# Define media sources
local_audio_short = "test_media/10_seconds.wav"  # 10 seconds
local_audio_long = "test_media/5_minutes.mp3"  # 5 minutes
youtube_audio = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"  # Rick Astley - Never Gonna Give You Up
local_image1 = "test_media/square.jpg" # square
local_image2 = "16_9.png" # 16:9
local_video_short = "test_media/15_seconds.mp4"  # 15 seconds
local_video_short2 = "test_media/15_seconds2.mp4"  # 15 seconds
local_video_long = "test_media/3_minutes.webm"  # 3 minutes
youtube_video = "https://www.youtube.com/watch?v=9bZkp7q19f0"  # PSY - GANGNAM STYLE
bg_music = "test_media/IndigoMirage.mp3"

# Text for text-to-speech generation
tts_text = "To be, or not to be, that is the question."

## Test 1: Single Image with Main Audio

Expected behavior: The image should be displayed for the entire duration of the audio.

In [2]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--output", "test1.mp4"])
print(f"Test 1 {'passed' if success else 'failed'}")
display_output(output_file)

get_audio_duration: file_path=test_media/10_seconds.wav, output='10.000340'
FFmpeg command:
ffmpeg -y -i temp_assets\temp_square.jpg.mp4 -i test_media/10_seconds.wav -filter_complex [0:v]fps=30,format=yuv420p,trim=duration=12.50034,setpts=PTS-STARTPTS[v0];[v0]fade=t=out:st=10.50034:d=2.0[final_video];[1:a]adelay=500|500,apad=pad_dur=2.0[main_audio];[main_audio]acopy[final_audio] -map [final_video] -map [final_audio] -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 192k -t 12.50034 test_output_test1.mp4
Video created successfully at test_output_test1.mp4
The length of the video is the main audio length plus 2.5 seconds.
Cleaning up temporary files...
Attempting to clean up 1 files:
Removed temporary file: temp_assets\temp_square.jpg.mp4
Cleanup completed. Temporary folder 'temp_assets' was not removed.

ffmpeg version 2024-05-13-git-37db0454e4-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --

## Test 2: Single Image without Main Audio

Expected behavior: The image should be displayed for 5 seconds.

In [3]:
success, output_file = run_mmmeld(["--image", local_image1, "--output", "test2.mp4"])
print(f"Test 2 {'passed' if success else 'failed'}")
display_output(output_file)

FFmpeg command:
ffmpeg -y -i temp_assets\temp_square.jpg.mp4 -filter_complex [0:v]fps=30,format=yuv420p,setpts=PTS-STARTPTS[final_video] -map [final_video] -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 192k test_output_test2.mp4
Video created successfully at test_output_test2.mp4
The length of the video is determined by the input images and videos.
Cleaning up temporary files...
Attempting to clean up 1 files:
Removed temporary file: temp_assets\temp_square.jpg.mp4
Cleanup completed. Temporary folder 'temp_assets' was not removed.

ffmpeg version 2024-05-13-git-37db0454e4-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-l

## Test 3: Single Video with Main Audio (Video Shorter)

Expected behavior: The video should loop to match the audio duration.

In [4]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", local_video_short, "--output", "test3.mp4"])
print(f"Test 3 {'passed' if success else 'failed'}")
display_output(output_file)

get_audio_duration: file_path=test_media/5_minutes.mp3, output='299.546122'
get_media_duration: file_path=test_media/15_seconds.mp4, output='15.000000'
FFmpeg command:
ffmpeg -y -i test_media/15_seconds.mp4 -i test_media/5_minutes.mp3 -filter_complex [0:v]fps=30,format=yuv420p,trim=duration=302.046122,setpts=PTS-STARTPTS[v0];[v0]fade=t=out:st=300.046122:d=2.0[final_video];[1:a]adelay=500|500,apad=pad_dur=2.0[main_audio];[main_audio]acopy[final_audio] -map [final_video] -map [final_audio] -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 192k -t 302.046122 test_output_test3.mp4
Video created successfully at test_output_test3.mp4
The length of the video is the main audio length plus 2.5 seconds.
Cleaning up temporary files...
Attempting to clean up 0 files:
Cleanup completed. Temporary folder 'temp_assets' was not removed.

ffmpeg version 2024-05-13-git-37db0454e4-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)


## Test 4: Single Video with Main Audio (Video Longer)

Expected behavior: The video should be cut to match the audio duration.

In [6]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_video_long, "--output", "test4.mp4"])
print(f"Test 4 {'passed' if success else 'failed'}")
display_output(output_file)

get_audio_duration: file_path=test_media/10_seconds.wav, output='10.000340'
get_media_duration: file_path=test_media/3_minutes.webm, output='180.208000'
FFmpeg command:
ffmpeg -y -i test_media/3_minutes.webm -i test_media/10_seconds.wav -filter_complex [0:v]fps=30,format=yuv420p,loop=1:size=5406[looped_video];[looped_video]trim=duration=12.50034,setpts=PTS-STARTPTS[v0];[v0]fade=t=out:st=10.50034:d=2.0[final_video];[1:a]adelay=500|500,apad=pad_dur=2.0[main_audio];[main_audio]acopy[final_audio] -map [final_video] -map [final_audio] -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 192k -t 12.50034 test_output_test4.mp4
Attempting to clean up 0 files:
Cleanup completed. Temporary folder 'temp_assets' was not removed.
Video created successfully at test_output_test4.mp4
The length of the video is the main audio length plus 2.5 seconds.
Cleaning up temporary files...
Attempting to clean up 0 files:
Cleanup completed. Temporary folder 'temp_assets' was not removed.

ffmpeg version 2024-05-13-

## Test 5: Single Video without Main Audio

Expected behavior: The video should play once with its own audio.

In [7]:
success, output_file = run_mmmeld(["--image", local_video_short2, "--output", "test5.mp4"])
print(f"Test 5 {'passed' if success else 'failed'}")
display_output(output_file)

get_media_duration: file_path=test_media/15_seconds2.mp4, output='15.162630'
get_media_duration: file_path=test_media/15_seconds2.mp4, output='15.162630'
FFmpeg command:
ffmpeg -y -i test_media/15_seconds2.mp4 -filter_complex [0:v]fps=30,format=yuv420p,loop=1:size=454[looped_video];[looped_video]setpts=PTS-STARTPTS[final_video];[0:a]aformat=sample_fmts=fltp:sample_rates=44100:channel_layouts=stereo[video_audio];[video_audio]acopy[final_audio] -map [final_video] -map [final_audio] -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 192k test_output_test5.mp4
Attempting to clean up 0 files:
Cleanup completed. Temporary folder 'temp_assets' was not removed.
Video created successfully at test_output_test5.mp4
The length of the video is determined by the input images and videos.
Cleaning up temporary files...
Attempting to clean up 0 files:
Cleanup completed. Temporary folder 'temp_assets' was not removed.

ffmpeg version 2024-05-13-git-37db0454e4-full_build-www.gyan.dev Copyright (c) 2000-20

## Test 6: Multiple Videos with Main Audio (Total Video Time <= Audio Time)

Expected behavior: Videos should play in sequence, then loop.

In [8]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_video_short},{local_video_short2}", "--output", "test6.mp4"])
print(f"Test 6 {'passed' if success else 'failed'}")
display_output(output_file)

get_audio_duration: file_path=test_media/5_minutes.mp3, output='299.546122'
get_media_duration: file_path=test_media/15_seconds.mp4, output='15.000000'
get_media_duration: file_path=test_media/15_seconds2.mp4, output='15.162630'
FFmpeg command:
ffmpeg -y -i test_media/15_seconds.mp4 -i test_media/15_seconds2.mp4 -i test_media/5_minutes.mp3 -filter_complex [0:v]fps=30,format=yuv420p,loop=11:size=904[looped_video];[looped_video]trim=duration=302.046122,setpts=PTS-STARTPTS[v0];[v0]fade=t=out:st=300.046122:d=2.0[final_video];[1:a]adelay=500|500,apad=pad_dur=2.0[main_audio];[main_audio]acopy[final_audio] -map [final_video] -map [final_audio] -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 192k -t 302.046122 test_output_test6.mp4
Attempting to clean up 0 files:
Cleanup completed. Temporary folder 'temp_assets' was not removed.
Video created successfully at test_output_test6.mp4
The length of the video is the main audio length plus 2.5 seconds.
Cleaning up temporary files...
Attempting to c

## Test 7: Multiple Videos with Main Audio (Total Video Time > Audio Time)

Expected behavior: Videos should play in sequence, cut at audio end.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_video_short},{local_video_long}", "--output", "test7.mp4"])
print(f"Test 7 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 8: Multiple Videos without Main Audio

Expected behavior: Videos should play in sequence once.

In [ ]:
success, output_file = run_mmmeld(["--image", f"{local_video_short},{local_video_long}", "--output", "test8.mp4"])
print(f"Test 8 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 9: Videos + Images with Main Audio (Total Video Time < Audio Time)

Expected behavior: Play videos once in sequence, distribute remaining time equally among images.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_video_short},{local_image1},{local_image2}", "--output", "test9.mp4"])
print(f"Test 9 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 10: Videos + Images with Main Audio (Total Video Time >= Audio Time)

Expected behavior: Distribute 5 seconds each to images, truncate sequence to fit audio.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_video_long},{local_image1},{local_image2}", "--output", "test10.mp4"])
print(f"Test 10 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 11: Videos + Images without Main Audio

Expected behavior: Play videos in sequence, show each image for 5 seconds.

In [ ]:
success, output_file = run_mmmeld(["--image", f"{local_video_short},{local_image1},{local_video_long},{local_image2}", "--output", "test11.mp4"])
print(f"Test 11 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 12: Image + Video with Main Audio (Video Shorter)

Expected behavior: End of video anchored to end of audio, image shown first, filling remaining time.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_image1},{local_video_short}", "--output", "test12.mp4"])
print(f"Test 12 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 13: Image + Video with Main Audio (Video Longer)

Expected behavior: Minimum 5s for image, then video (cut off).

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_image1},{local_video_long}", "--output", "test13.mp4"])
print(f"Test 13 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 14: Image + Video without Main Audio

Expected behavior: Image shown for 5 seconds, then video plays in full.

In [ ]:
success, output_file = run_mmmeld(["--image", f"{local_image1},{local_video_long}", "--output", "test14.mp4"])
print(f"Test 14 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 15: Multiple Images with Main Audio

Expected behavior: Equal time for each image, no looping.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_image1},{local_image2}", "--output", "test15.mp4"])
print(f"Test 15 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 16: Multiple Images without Main Audio

Expected behavior: 5 seconds for each image.

In [ ]:
success, output_file = run_mmmeld(["--image", f"{local_image1},{local_image2}", "--output", "test16.mp4"])
print(f"Test 16 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 17: YouTube Audio with Local Images

Expected behavior: YouTube audio with images displayed for equal durations.

In [ ]:
success, output_file = run_mmmeld(["--audio", youtube_audio, "--image", f"{local_image1},{local_image2}", "--output", "test17.mp4"])
print(f"Test 17 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 18: Local Audio with YouTube Video

Expected behavior: Local audio with YouTube video (cut or looped to match audio duration).

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", youtube_video, "--output", "test18.mp4"])
print(f"Test 18 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 19: Text-to-Speech with AI-Generated Image

Expected behavior: Generated speech with AI-generated image for the full duration.

In [ ]:
success, output_file = run_mmmeld(["--text", tts_text, "--image", "generate", "--output", "test19.mp4"])
print(f"Test 19 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 20: Main Audio with Background Music

Expected behavior: Main audio plays with background music looped and faded out at the end.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--bg-music", bg_music, "--output", "test20.mp4"])
print(f"Test 20 {'passed' if success else 'failed'}")
display_output(output_file)

## Test 21: Custom Audio Margins

Expected behavior: 1-second lead-in and 3-second fade-out applied to the main audio.

In [ ]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--audiomargin", "1.0,3.0", "--output", "test21.mp4"])
print(f"Test 21 {'passed' if success else 'failed'}")
display_output(output_file)